## Using prompt engineering to evaluate reviews

For this practive we want to use prompt engineering and large-language models to evaluate the review text that we collected for Taco Shops from Yelp in our HW3.

The first part is just collect the reviews again, which we know how to do it already.

In [ ]:
# Libraries
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
!pip install openai
!pip install yelpapi

In [ ]:
# Now enter your API key so you can make requests
from yelpapi import YelpAPI
# yelp_api = YelpAPI('ENTER YOUR KEY HERE')
api_key = 'YOUR-YELP-API-KEY'
yelp_api = YelpAPI(api_key)


In [ ]:
# search for taco shops and store to taco_shops
taco_shops = yelp_api.search_query(term = 'taco',  location = 'Tucson, AZ')

In [ ]:
# Flatten to taco_shops_df
taco_shops_df = pd.json_normalize(taco_shops['businesses'])


In [ ]:
# Select only necessary columns
taco_shops_df = taco_shops_df[['id', 'alias', 'name', 'review_count', 'rating']]

In [ ]:
# Getting reviews
import time
taco_shops_reviews_df = list()

for i in range(taco_shops_df.shape[0]):
  reviews = yelp_api.reviews_query(id = taco_shops_df['id'][i])
  reviews_df = pd.json_normalize(reviews, record_path='reviews')
  reviews_df['location_id'] = taco_shops_df['id'][i]
  taco_shops_reviews_df.append(reviews_df)
taco_shops_reviews_df = pd.concat(taco_shops_reviews_df)

In [ ]:
taco_shops_reviews_df = taco_shops_reviews_df[['id', 'text', 'rating', 'time_created', 'location_id']]

In [ ]:
#Setup prompt
from openai import OpenAI

#to get the OPENAI_API_KEY you must create account with openai.com and then use API (not chatGPT)
client = OpenAI(
    api_key='YOUR-OPEN-AI-API-KEY',
)


def get_completion(prompt, model="gpt-4o", temperature=0):
    chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model=model,
      temperature=temperature)
    return chat_completion.choices[0].message.content


In context learning: When we want to use large-language model to solve a new task, we can create a set of example to demonstrate the task. For exmple, here we provide couple of reviews and the sentiment and ask the language model to provide the sentiment for the given review.

In [ ]:

template1 = """Classify the text into neutral, negative or positive.
Text: ```{review}```
Sentiment:"""

#Use examples to guide LLM in template 2
template2 = """Classify the text into neutral, negative or positive.

#add your examples here

Text: ```{review}```
Sentiment:"""


def format_prompt(template, review):
  return template.format(review=review);

Let's test it with a simple example

In [ ]:
prompt = format_prompt(template1, "my food was not good")
response = get_completion(prompt)
print(response)

Now, we create a prompt for each review and pass to the GPT-4o large-language model to determine the sentiment of the review.

In [ ]:
#Complete the loop
review_sentiment = list()
for i in range(taco_shops_reviews_df.shape[0]):
  ...

In [ ]:
#Assign the result as the new column of taco_shops_reviews_df called sentiment
...

Note the YelpAPI gives us truncated review text, so the sentiment that we get may not be consistent with the provided rating.


In [ ]:
#check
taco_shops_reviews_df

In [ ]:
#Now use template2 and compare the output


Can you modify the template to get a better output? How about adding more examples in the prompt? Write templete3 and use that.